In [52]:
import numpy as np
import pandas as pd

In [53]:
#Reading Data
train_set = pd.read_csv('../../Datasets/Housingdata/Housing_Train.csv')
test_set = pd.read_csv('../../Datasets/Housingdata/Housing_Test.csv')
train_set

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.25199,0.0,10.59,0.0,0.489,5.783,72.7,4.3549,4,277,18.6,389.43,NaN,22.5
1,0.62976,0.0,8.14,0.0,0.538,5.949,61.8,4.7075,4,307,21.0,396.90,8.26,20.4
2,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4,307,21.0,396.90,18.72,15.2
3,NaN,0.0,18.10,1.0,0.631,6.683,96.8,1.3567,24,666,20.2,375.33,3.73,50.0
4,12.80230,0.0,18.10,0.0,0.740,5.854,96.6,1.8956,24,666,20.2,240.52,23.79,10.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,1.00245,0.0,8.14,0.0,0.538,6.674,87.3,4.2390,4,307,21.0,380.23,11.98,21.0
401,0.03041,0.0,5.19,0.0,0.515,5.895,59.6,5.6150,5,224,20.2,394.81,10.56,18.5
402,2.81838,0.0,18.10,0.0,0.532,5.762,40.3,4.0983,24,666,20.2,392.92,10.42,21.8
403,0.44791,0.0,6.20,1.0,0.507,6.726,66.5,3.6519,8,307,17.4,360.20,8.05,29.0


In [54]:
#splliting datasets
y_train = train_set['MEDV']
x_train = train_set.drop(columns='MEDV')

x_test = test_set

In [55]:
#check dataset
x_train.isnull().sum()
x_test.isnull().sum()

CRIM       3
ZN         5
INDUS      6
CHAS       4
NOX        0
RM         0
AGE        5
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      2
dtype: int64

In [56]:
#fill Null values
def replace_null_values(cols: list, dataset: pd.DataFrame):
    for col in cols:
        temp = dataset[col].dropna()
        mode = temp.mean()
        dataset[col].fillna(mode, inplace=True)

na_cols = list(x_train.columns[x_train.isnull().any()])
na_cols_test = list(x_test.columns[x_test.isnull().any()])

replace_null_values(na_cols, x_train)
replace_null_values(na_cols_test, x_test)

In [57]:
#feature Scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train), columns=x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns=x_test.columns)

In [58]:
# Linear Regression
class LinearRegression:
    def __init__(self):
        self.weight = None

    def fit(self, x:pd.DataFrame, y:pd.DataFrame, epoch:int, learning_rate:float):
        m = len(x)
        if x.iloc[0, 0] != 1:
            x.insert(0, 'bias', 1)

        self.weight = np.ones(len(x.columns))

        for i in range(epoch):
            gradient = 2/m * x.T.dot(self.predict(x) - y)
            self.weight = self.weight - learning_rate * gradient

    def predict(self, df:pd.DataFrame):
        if df.iloc[0, 0] != 1:
            df.insert(0, 'bias', 1)
        return np.dot(df, self.weight)

In [77]:
reg = LinearRegression()
reg.fit(x_train, y_train, 2000, 0.1)

In [78]:
# Evaluating Model
from sklearn.metrics import mean_squared_error, r2_score
y_pred_train = reg.predict(x_train)

mse_error = mean_squared_error(y_train, y_pred_train)
rmse = np.sqrt(mse_error)
r2_error = r2_score(y_train, y_pred_train)

errors = pd.DataFrame({'Value': [mse_error, rmse, r2_error]}, index=['MSE', 'RMSE', 'R2'])
errors

,Value
MSE,22.446441
RMSE,4.737767
R2,0.738856


In [79]:
y_pred_train

array([20.20593033, 19.62213737, 16.4375262 , 31.42168278, 11.38362399,
       30.66857533, 25.39306017, 10.85345345, 23.94356721,  2.5205598 ,
       11.63152702, 32.83158877, 12.23275816, 18.70320064, 27.60595805,
       20.7991362 , 26.38847883, 42.12750974, 27.46493651, 31.34994328,
       17.253358  , 25.20061732, 37.35189381, 12.17574213, 25.42977225,
       27.89873077, 10.93418248,  5.5886609 , 21.4412389 ,  9.27286088,
       24.72485304, 31.24319432, 14.09697101, 12.99211693, 20.71611679,
       19.61200123, 26.16840354, 17.98431314, 18.17255847, 24.80760856,
       17.92684111, 35.03156337, 21.83785856, 21.00454849, 20.7217062 ,
        0.62557738, 22.46448125, 17.36703514, 14.08729572, 42.81465004,
        9.9602024 , 33.3885066 , 16.18411832, 20.87253774, 23.02885621,
       24.28956763, 19.50659509, 19.07266124, 20.05289647, 30.99788054,
       20.9589002 , 17.56725521, 14.72040504, 19.87674131, 14.47638405,
       17.77254798, 20.01914103, 13.20890284, 17.47148158, 28.21